Парсер уцененных товаров с сайта dns-shop.ru
Код частично удален, полный код можно запросить тут: 9m6@bk.ru 

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import re

In [2]:
URL = 'https://www.dns-shop.ru/catalog/markdown/?p='
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0', 'accept': '*/*'}
FILE = 'dns_ucenka_.csv'
HOST = 'https://www.dns-shop.ru'

In [3]:
def get_html(url, params=None):
    request = requests.get(url, headers=HEADERS, params=params)
    return request
get_html_URL = get_html(URL).text
get_html_URL

'\n<!DOCTYPE html>\n<html lang="ru">\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n\t<meta charset="UTF-8" />\n\t<meta name="csrf-param" content="_csrf">\n<meta name="csrf-token" content="xFGzIaO02gP9V_WbHv-6woA74qJnmqdSgPkUzqcptUusB8MWzMe1aYVkwaxSieW4-Wmmj1_U9ifBtnalyxDzBA==">\n\t<link rel="shortcut icon" href="/assets/fc87bcf/images/theme/favicon.png" type="image/png" />\n\t<title>Уценённые товары | Каталог | DNS - DNS – интернет магазин цифровой и бытовой техники по доступным ценам.</title>\n<link rel="manifest" href="/files/manifest/manifest.json" />\n\n<meta name="theme-color" content="#F6F6F6" />\n<meta name="application-name" content="DNS" />\n<meta name="mobile-web-app-capable" content="yes" />\n<meta name="viewport" content="width=device-width, initial-scale=1" />\n\n<link rel="icon" type="image/png" sizes="16x16" href="/files/manifest/favicon-16x16.png" />\n<link rel="icon" type="image/png" sizes="24x24" href="/files/manifest/favicon-24x24.png" />\n<link

In [5]:
def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='catalog-product')
    schools = []
    for item in items:
        old_price = item.find('div', class_='catalog-product__price-old')
        if old_price is None:
            old_price = 1
        else:
            old_price = int(old_price.get_text(strip=True).strip(' ₽').replace(' ',''))
        price = item.find('div', class_='catalog-product__price-actual')
        if price is None:
            price = -1
        else:
            price = int(price.get_text(strip=True).strip(' ₽').replace(' ',''))            
        schools.append({
            'title': item.find('a', class_='catalog-product__name ui-link ui-link_black').get_text(strip=True),
            'link': HOST + item.find('a', class_='catalog-product__name ui-link ui-link_black').get('href'),
            'price': price,
            'old_price': old_price,
            'benefit': old_price - price,
            'procent': round(100 - (price/old_price)*100)
        })
    return schools

In [6]:
def save_file(items, path):
    with open(path, 'w', newline='', errors='ignore') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['Товар', 'Ссылка', 'Цена','Старая цена','Выгода','Процент'])
        for item in items:
            writer.writerow([item['title'], item['link'], item['price'],
                             item['old_price'], item['benefit'], item['procent']])